In [ ]:
import os
import pandas as pd
import numpy as np
import h5py
import torch
from transformers import AutoTokenizer
from scipy.stats import spearmanr
from sklearn.metrics import roc_auc_score

import sys
sys.path.insert(0, "/cluster/pixstor/xudong-lab/yuexu/SeqDance-main/SeqDance-main/model/")
from config import config # please first download the dataset and fill in the config.py file with the path where you downloaded the dataset
from model import ESMwrap

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t12_35M_UR50D")

# Load model
esm2_select = 'model_35M'
model_select = 'seqdance' # or 'esmdance'
dance_model = ESMwrap(esm2_select, model_select).to(device)

# Load the SeqDance model from huggingface
dance_model = dance_model.from_pretrained("ChaoHou/ESMDance")
dance_model = dance_model.to(device)
dance_model.eval()

In [ ]:
df = pd.read_csv('/cluster/pixstor/xudong-lab/yuexu/SeqDance-main/SeqDance-main/data/Processed_K50_dG_datasets/Processed_K50_dG_datasets/Tsuboyama2023_Dataset2_Dataset3_20230416.csv')

# data filtering: 1. have ddG value, 2. not insertion or deletion as the sequence lengths are different
df = df[(df['ddG_ML']!='-') & (~df['mut_type'].str.contains('ins|del'))]
df['ddG_ML'] = df['ddG_ML'].astype(float)

# get the mean ddG value for each aa_seq (some aa_seq have multiple experiments)
ddg_mean = df[['aa_seq','ddG_ML']].groupby('aa_seq').mean()
df = pd.merge(df.sort_values('mut_type', ascending=False).drop_duplicates('aa_seq')[['aa_seq','mut_type','WT_name','WT_cluster']], ddg_mean, on='aa_seq', how='left')

df.index = df['WT_name'] + '$' + df['mut_type']

pro = 'r10_572_TrROS_Hall.pdb'
df_pro = df[df['WT_name'] == pro]

# get the prediction for the wildtype sequence
wt_seq = df.loc[f'{pro}$wt','aa_seq']
wt_input = tokenizer(wt_seq, return_tensors="pt").to(device)
with torch.no_grad():
    wt_output = dance_model(wt_input)

# all mutations
pro_muts = df[df['WT_name']==pro]['mut_type']

# the epsilon is used to avoid division by zero
epsilon = 1e-2

for mt in pro_muts:
    # get the prediction for the mutant sequence
    mt_seq = df.loc[f'{pro}${mt}','aa_seq']
    mt_input = tokenizer(mt_seq, return_tensors="pt").to(device)
    with torch.no_grad():
        mt_output = dance_model(mt_input)
    # calculate the relative difference between the mutant and wildtype predictions
    for feature in wt_output:
        df_pro.loc[f'{pro}${mt}',feature] = (abs(mt_output[feature] - wt_output[feature]) / (abs(wt_output[feature]) + epsilon)).mean().item()

In [ ]:
def quantile_normalization(features):
    """
    Applies quantile normalization to a set of feature vectors (each column represents a protein).
    Ensures all features have the same distribution across proteins.
    """
    features = np.array(features)
    
    # Rank transformation
    ranks = np.argsort(np.argsort(features, axis=0), axis=0)
    
    # Compute mean per rank across all features
    sorted_features = np.sort(features, axis=0)
    rank_means = np.mean(sorted_features, axis=1)
    
    # Map original values to rank means
    normalized_matrix = np.zeros_like(features, dtype=np.float64)
    for i in range(features.shape[1]):
        normalized_matrix[:,i] = rank_means[ranks[:,i]]
    
    return normalized_matrix

def get_normalized_values(df, features):
    """
    Normalizes the values of the features to have the same distribution across proteins.
    """
    normalized_value = quantile_normalization(df[features].values)
    normalized_df = pd.DataFrame(normalized_value, columns=[i + '_norm' for i in features])
    normalized_df.index = df.index
    df = pd.merge(df, normalized_df, left_index=True, right_index=True)
    return df

In [ ]:
features = df_pro.columns[5:]
df_pro = get_normalized_values(df_pro, features)

# Convert to NumPy array
raw_features = df_pro[features].values
normalized_features = df_pro[[i + '_norm' for i in features]].values

# Compute row-wise weights (value / sum of row)
raw_weights = (raw_features + 1e-8) / (raw_features + 1e-8).sum(axis=1, keepdims=True)
norm_weights = (normalized_features + 1e-8) / (normalized_features + 1e-8).sum(axis=1, keepdims=True)

# Compute feature combinations
df_pro['raw_mean'] = raw_features.mean(axis=1)
df_pro['raw_max'] = raw_features.max(axis=1)
df_pro['raw_weighted_mean'] = np.sum(raw_features * raw_weights, axis=1)
df_pro['raw_geometric_mean'] = np.exp(np.mean(np.log(raw_features + 1e-8), axis=1))

df_pro['norm_mean'] = normalized_features.mean(axis=1)
df_pro['norm_max'] = normalized_features.max(axis=1)
df_pro['norm_weighted_mean'] = np.sum(normalized_features * norm_weights, axis=1)
df_pro['norm_geometric_mean'] = np.exp(np.mean(np.log(normalized_features + 1e-8), axis=1))

df_pro.iloc[:, 4:].corr(method='spearman')

In [ ]:
import esm
# Load the ESM-2 model
esm_35, alphabet = esm.pretrained.esm2_t12_35M_UR50D()
esm_35 = esm_35.to(device)
esm_35.eval()
# Create a batch converter
batch_converter = alphabet.get_batch_converter()

# use wildtype sequence to get the logits, then get LLR, this perform similar to masked LLR
def get_wt_logits(esm_model, batch_converter, seq):
    batch_labels, batch_strs, batch_tokens = batch_converter([('name', seq)])
    with torch.no_grad():
        results = esm_model(batch_tokens.to(device))
    logits = torch.log_softmax(results["logits"],dim=-1)[0,:,:].cpu().numpy()[1:-1,:]
    return logits

# mask each residue, get the logits of masked residue, then get LLR
def get_mask_llr(esm_model, batch_converter, alphabet, seq):
    batch_labels, batch_strs, batch_tokens = batch_converter([('name', seq)])
    mask_logits = []
    for i in range(len(seq)):
        batch_tokens_masked = batch_tokens.clone()
        batch_tokens_masked[0, i+1] = alphabet.mask_idx
        with torch.no_grad():
            results = esm_model(batch_tokens_masked.to(device))
        mask_logits.append(results["logits"][:,i+1,:])
    
    mask_logits = torch.cat(mask_logits, dim=0)
    mask_logits = torch.log_softmax(mask_logits,dim=-1).cpu().numpy()

    return mask_logits

def logits_to_llr(logits, alphabet, seq):
    logits = pd.DataFrame(logits,columns=alphabet.all_toks,index=list(seq)).T
    wt_norm=np.diag(logits.loc[logits.columns])
    llr = logits-wt_norm
    
    llr.columns = [seq[i] + str(i+1) for i in range(len(seq))]
    
    llr = pd.DataFrame(llr.iloc[4:24].T.stack(), columns=['LLR']).reset_index()
    llr['mutant'] = llr['level_0'].str.replace('_','') + llr['level_1']
    return llr[['mutant','LLR']].set_index('mutant')

# get the zero-shot LLR using both wildtype and masked logits
wt_logits = get_wt_logits(esm_35, batch_converter, wt_seq)
wt_llr = logits_to_llr(wt_logits, alphabet, wt_seq)

mask_logits = get_mask_llr(esm_35, batch_converter, alphabet, wt_seq)
mask_llr = logits_to_llr(mask_logits, alphabet, wt_seq)
for mt in pro_muts:
    if mt != 'wt':
        # merge the LLR values if there are multiple mutations
        df_pro.loc[f'{pro}${mt}', 'wt_llr'] = wt_llr.loc[mt.split(':'), 'LLR'].sum()
        df_pro.loc[f'{pro}${mt}', 'mask_llr'] = mask_llr.loc[mt.split(':'), 'LLR'].sum()
    else:
        df_pro.loc[f'{pro}${mt}', 'wt_llr'] = 0
        df_pro.loc[f'{pro}${mt}', 'mask_llr'] = 0

df_pro.iloc[:,4:].corr(method='spearman')

In [ ]:
#####################DPLM

In [ ]:
import argparse

import torch

import esm
import esm_adapterH

import numpy as np
import yaml
from utils.utils import load_configs
from collections import OrderedDict
from Bio.PDB import PDBParser, PPBuilder
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import pairwise_distances
import os
import h5py
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from peft import PeftModel, LoraConfig, get_peft_model

def load_checkpoints(model,checkpoint_path):
        if checkpoint_path is not None:
            checkpoint = torch.load(checkpoint_path, map_location=lambda storage, loc: storage)
            if  any('adapter' in name for name, _ in model.named_modules()):
                if np.sum(["adapter_layer_dict" in key for key in checkpoint[
                    'state_dict1'].keys()]) == 0:  # using old checkpoints, need to rename the adapter_layer into adapter_layer_dict.adapter_0
                    new_ordered_dict = OrderedDict()
                    for key, value in checkpoint['state_dict1'].items():
                        if "adapter_layer_dict" not in key:
                            new_key = key.replace('adapter_layer', 'adapter_layer_dict.adapter_0')
                            new_ordered_dict[new_key] = value
                        else:
                            new_ordered_dict[key] = value
                    
                    model.load_state_dict(new_ordered_dict, strict=False)
                else:
                    #this model does not contain esm2
                    new_ordered_dict = OrderedDict()
                    for key, value in checkpoint['state_dict1'].items():
                            key = key.replace("esm2.","")
                            new_ordered_dict[key] = value
                    
                    model.load_state_dict(new_ordered_dict, strict=False)
            elif  any('lora' in name for name, _ in model.named_modules()):
                #this model does not contain esm2
                new_ordered_dict = OrderedDict()
                for key, value in checkpoint['state_dict1'].items():
                        print(key)
                        key = key.replace("esm2.","")
                        new_ordered_dict[key] = value
                
                model.load_state_dict(new_ordered_dict, strict=False)
            
            print("checkpoints were loaded from " + checkpoint_path)
        else:
            print("checkpoints not exist "+ checkpoint_path)

# def load_model(args):
#     if args.model_type=="d-plm":
#         with open(args.config_path) as file:
#             config_file = yaml.full_load(file)
#             configs = load_configs(config_file, args=None)
        
#         # inference for each model
#         model, alphabet = esm_adapterH.pretrained.esm2_t33_650M_UR50D(configs.model.esm_encoder.adapter_h)
#         load_checkpoints(model,args.model_location)
#     elif args.model_type=="ESM2":
#         #if use ESM2
#         model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
    
    
#     model.eval()
#     if torch.cuda.is_available():
#         model = model.cuda()
#         print("Transferred model to GPU")
    
#     return model,alphabet

def load_model(args):
    if args.model_type=="d-plm":
        with open(args.config_path) as file:
            config_file = yaml.full_load(file)
            configs = load_configs(config_file, args=None)
        if configs.model.esm_encoder.adapter_h.enable:
            model, alphabet = esm_adapterH.pretrained.esm2_t33_650M_UR50D(configs.model.esm_encoder.adapter_h)
        elif configs.model.esm_encoder.lora.enable:
            model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
            lora_targets =  ["self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj","self_attn.out_proj"]
            target_modules=[]
            if configs.model.esm_encoder.lora.esm_num_end_lora > 0:
                start_layer_idx = np.max([model.num_layers - configs.model.esm_encoder.lora.esm_num_end_lora, 0])
                for idx in range(start_layer_idx, model.num_layers):
                    for layer_name in lora_targets:
                        target_modules.append(f"layers.{idx}.{layer_name}")
                
            peft_config = LoraConfig(
                inference_mode=False,
                r=configs.model.esm_encoder.lora.r,
                lora_alpha=configs.model.esm_encoder.lora.alpha,
                target_modules=target_modules,
                lora_dropout=configs.model.esm_encoder.lora.dropout,
                bias="none",
                # modules_to_save=modules_to_save
            )
            peft_model = get_peft_model(model, peft_config)
        
        # inference for each model
        # model, alphabet = esm_adapterH.pretrained.esm2_t33_650M_UR50D(configs.model.esm_encoder.adapter_h)/
        load_checkpoints(model,args.model_location)
    elif args.model_type=="ESM2":
        #if use ESM2
        model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
    
    
    model.eval()
    if torch.cuda.is_available():
        model = model.cuda()
        print("Transferred model to GPU")
    
    return model,alphabet

def main(args):
    batch_converter = args.alphabet.get_batch_converter()
    
    data = [
        ("protein1", args.sequence),
    ]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    
    with torch.no_grad():
        wt_representation = args.model(batch_tokens.cuda(),repr_layers=[args.model.num_layers])["representations"][args.model.num_layers]
    
    wt_representation = wt_representation.squeeze(0) #only one sequence a time
    return wt_representation

parser = argparse.ArgumentParser(description='PyTorch SimCLR')
parser.add_argument("--model-location", type=str, help="xx", default='/cluster/pixstor/xudong-lab/yuexu/D_PLM/results/vivit_3/checkpoints/checkpoint_best_val_rmsf_cor.pth')
parser.add_argument("--config-path", type=str, default='/cluster/pixstor/xudong-lab/yuexu/D_PLM/results/vivit_3/config_vivit3.yaml', help="xx")
parser.add_argument("--model-type", default='d-plm', type=str, help="xx")
parser.add_argument("--sequence", type=str, help="xx")
parser.add_argument("--output_path", type=str, default='/cluster/pixstor/xudong-lab/yuexu/D_PLM/evaluate/', help="xx")
args = parser.parse_args()

args.model,args.alphabet = load_model(args)

In [ ]:
############zero shot
pro_id_set=set()
for i in df.index:
    pro_id_set.add(i.split("$")[0])

corr_list=[]

for pro in pro_id_set:
    # pro = 'r10_572_TrROS_Hall.pdb'
    df_pro = df[df['WT_name'] == pro]
    # get the prediction for the wildtype sequence
    wt_seq = df.loc[f'{pro}$wt','aa_seq']
    # wt_input = tokenizer(wt_seq, return_tensors="pt").to(device)
    # with torch.no_grad():
    #     wt_output = dance_model(wt_input)
    args.sequence=wt_seq
    wt_output = main(args)
    wt_output = wt_output[1:-1]
    # all mutations
    pro_muts = df[df['WT_name']==pro]['mut_type']
    # the epsilon is used to avoid division by zero
    epsilon = 1e-2
    ddG_list=[]
    score_list=[]
    for mt in pro_muts:
        if mt=="wt":
            continue
        # get the prediction for the mutant sequence
        mt_seq = df.loc[f'{pro}${mt}','aa_seq']
        # mt_input = tokenizer(mt_seq, return_tensors="pt").to(device)
        args.sequence=mt_seq
        mt_output = main(args)
        mt_output = mt_output[1:-1]
        score = np.linalg.norm((mt_output-wt_output).to('cpu').detach().numpy(),axis=1)
        score = np.log(np.mean(score))*-1
        score_list.append(score)
        ddG_list.append(df.loc[f'{pro}${mt}','ddG_ML'])
    
    print("done")
    corr, _ = spearmanr(ddG_list, score_list)
    corr_list.append(corr)

np.mean(corr_list) #0.441

np.median(corr_list) #0.456


In [ ]:
# regression each pro 50% for train
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from scipy.stats import spearmanr
import torch
from tqdm import tqdm

pro_id_set=set()
for i in df.index:
    pro_id_set.add(i.split("$")[0])
    
# ============================================================================
# STANDALONE VERSION (copy-paste directly into your code)
# ============================================================================

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from scipy.stats import spearmanr
from tqdm import tqdm

# Set random seed
np.random.seed(42)

# Collect data
X_train_list = []
y_train_list = []
X_test_list = []
y_test_list = []

print("Processing proteins and collecting mutations...")
for pro in tqdm(list(pro_id_set)):
    # Get wild-type embedding
    wt_seq = df.loc[f'{pro}$wt', 'aa_seq']
    args.sequence = wt_seq
    wt_output = main(args)
    wt_emb = wt_output[1:-1].cpu().numpy()  # Remove BOS/EOS
    
    # Get all mutations (excluding wildtype)
    pro_muts = df[df['WT_name']==pro]['mut_type'].values
    pro_muts = [mt for mt in pro_muts if mt != "wt"]
    
    if len(pro_muts) == 0:
        continue
    
    # Shuffle and split mutations for this protein (50/50)
    pro_muts_shuffled = np.random.permutation(pro_muts)
    n_train = len(pro_muts_shuffled) // 2
    train_muts = pro_muts_shuffled[:n_train]
    test_muts = pro_muts_shuffled[n_train:]
    
    # Process training mutations
    for mt in train_muts:
        mt_seq = df.loc[f'{pro}${mt}', 'aa_seq']
        args.sequence = mt_seq
        mt_output = main(args)
        mt_emb = mt_output[1:-1].cpu().numpy()
        
        feature = np.mean(mt_emb - wt_emb, axis=0)
        ddG = df.loc[f'{pro}${mt}', 'ddG_ML']
        
        X_train_list.append(feature)
        y_train_list.append(ddG)
    
    # Process test mutations
    for mt in test_muts:
        mt_seq = df.loc[f'{pro}${mt}', 'aa_seq']
        args.sequence = mt_seq
        mt_output = main(args)
        mt_emb = mt_output[1:-1].cpu().numpy()
        
        feature = np.mean(mt_emb - wt_emb, axis=0)
        ddG = df.loc[f'{pro}${mt}', 'ddG_ML']
        
        X_test_list.append(feature)
        y_test_list.append(ddG)

# Convert to arrays
X_train = np.stack(X_train_list)
y_train = np.array(y_train_list)
X_test = np.stack(X_test_list)
y_test = np.array(y_test_list)

print(f"\\nTraining mutations: {len(y_train)}")
print(f"Test mutations: {len(y_test)}")

# Train and evaluate
model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_mae = mean_absolute_error(y_train, y_train_pred)
train_spearman, _ = spearmanr(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_spearman, _ = spearmanr(y_test, y_test_pred)

print("\\n" + "="*60)
print("RESULTS - Per-Protein Mutation Split")
print("="*60)
print(f"Training Set: MAE={train_mae:.4f}, Spearman={train_spearman:.4f}")
print(f"Test Set:     MAE={test_mae:.4f}, Spearman={test_spearman:.4f}")
print("="*60)

In [ ]:
# regression repeat 20 times

# ============================================================================
# MINIMAL STANDALONE VERSION (copy-paste into your code)
# ============================================================================

# Step 1: Cache embeddings ONCE
embeddings = {}
for pro in tqdm(list(pro_id_set), desc="Caching embeddings"):
    embeddings[pro] = {'mutations': {}}
    
    # Wildtype
    wt_seq = df.loc[f'{pro}$wt', 'aa_seq']
    args.sequence = wt_seq
    wt_output = main(args)
    # embeddings[pro]['wt'] = wt_output[1:-1].cpu().numpy()
    wt_emb = wt_output[1:-1].cpu().numpy()
    
    # Mutations
    for mt in df[df['WT_name']==pro]['mut_type'].values:
        if mt == "wt":
            continue
        mt_seq = df.loc[f'{pro}${mt}', 'aa_seq']
        args.sequence = mt_seq
        mt_output = main(args)
        # embeddings[pro]['mutations'][mt] = mt_output[1:-1].cpu().numpy()
        mt_emb = mt_output[1:-1].cpu().numpy()
        embeddings[pro]['mutations'][mt] = np.mean(mt_emb - wt_emb, axis=0)

# Step 2: Run 20 times
results = []
for run_i in tqdm(range(20), desc="Running 20 splits"):
    np.random.seed(42 + run_i)
    
    X_train, y_train, X_test, y_test = [], [], [], []
    
    for pro in list(pro_id_set):
        # wt_emb = embeddings[pro]['wt']
        pro_muts = [mt for mt in df[df['WT_name']==pro]['mut_type'].values if mt != "wt"]
        
        if len(pro_muts) == 0:
            continue
        
        # Split 50/50
        pro_muts_shuffled = np.random.permutation(pro_muts)
        n_train = len(pro_muts_shuffled) // 2
        
        for mt in pro_muts_shuffled[:n_train]:
            # mt_emb = embeddings[pro]['mutations'][mt]
            # X_train.append(np.mean(mt_emb - wt_emb, axis=0))
            X_train.append(embeddings[pro]['mutations'][mt])
            y_train.append(df.loc[f'{pro}${mt}', 'ddG_ML'])
        
        for mt in pro_muts_shuffled[n_train:]:
            # mt_emb = embeddings[pro]['mutations'][mt]
            # X_test.append(np.mean(mt_emb - wt_emb, axis=0))
            X_test.append(embeddings[pro]['mutations'][mt])
            y_test.append(df.loc[f'{pro}${mt}', 'ddG_ML'])
    
    # Train and evaluate
    X_train = np.stack(X_train)
    y_train = np.array(y_train)
    X_test = np.stack(X_test)
    y_test = np.array(y_test)
    
    model = LinearRegression().fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    spearman, _ = spearmanr(y_test, y_pred)
    
    results.append({'run': run_i, 'test_mae': mae, 'test_spearman': spearman})

# Step 3: Summarize
results_df = pd.DataFrame(results)
print(f"\\nTest MAE: {results_df['test_mae'].mean():.4f} ± {results_df['test_mae'].std():.4f}")
print(f"Test Spearman: {results_df['test_spearman'].mean():.4f} ± {results_df['test_spearman'].std():.4f}")
